In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict
from nltk.util import ngrams

In [9]:
# utilisait clean et results... je ne les ai pas
df = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv')
df2 = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv')

In [48]:
list_applications_toEval = list(df['application_number'])
list_applications_KS = list(df2['application_number'])
print(len(list_applications_toEval), len(list_applications_KS))
print(list_applications_toEval[0:3])
print(list_applications_KS[0:3])

19 507
[14994365, 15028028, 15040198]
[12929813, 12937961, 12983488]


In [18]:
mypath = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2016'
json16 = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(json16)
json16[1:5]

#pOUR LA ks -- les 5 années avant
mypath_1 = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2015/'
_1_files = [f for f in listdir(mypath_1) if isfile(join(mypath_1, f))]
mypath_2 = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2014/'
_2_files = [f for f in listdir(mypath_2) if isfile(join(mypath_2, f))]
mypath_3 = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2013/'
_3_files = [f for f in listdir(mypath_3) if isfile(join(mypath_3, f))]
mypath_4 = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2012/'
_4_files = [f for f in listdir(mypath_4) if isfile(join(mypath_4, f))]
mypath_5 = 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data/2011/'
_5_files = [f for f in listdir(mypath_5) if isfile(join(mypath_5, f))]

In [19]:
def return_year(file_path):
    #print(file_path)
    if file_path in _1_files:
        return mypath_1
    if file_path in _2_files:
        return mypath_2
    if file_path in _3_files:
        return mypath_3
    if file_path in _4_files:
        return mypath_4
    if file_path in _5_files:
        return mypath_5

In [20]:
def pairing(list_ipc):
    pairs = []
    for L in range(len(list_ipc) + 1):
        for subset in itertools.combinations(list_ipc, L):
            if len(subset) == 2:
                pairs.append(subset)
    return pairs

In [42]:
i=2
print(list_applications_KS[i])
patent_path = str(list_applications_KS[i]) + '.json'
correct_path = return_year(patent_path)
patent_path = correct_path + patent_path
with open(patent_path) as f:
        d = json.load(f)
        f.close()
combi_IPC = [d["main_ipcr_label"]] + d["ipcr_labels"]
print("Main IPC: ", d["main_ipcr_label"])
print("Secondary IPCs: ", d["ipcr_labels"])
print("Combination: ", combi_IPC)

print("Pairing: ", pairing(combi_IPC)) # pairing renvoie toutes les pairs (même en double!)

combi_mainIPC = []
for class_ipc in d["ipcr_labels"]:
            combi_mainIPC.append((d["main_ipcr_label"], class_ipc))
print("Main+other: ", combi_mainIPC)


12983488
Main IPC:  G06F116
Secondary IPCs:  ['G06F116', 'G06F126', 'G06F1516']
Combination:  ['G06F116', 'G06F116', 'G06F126', 'G06F1516']
Pairing:  [('G06F116', 'G06F116'), ('G06F116', 'G06F126'), ('G06F116', 'G06F1516'), ('G06F116', 'G06F126'), ('G06F116', 'G06F1516'), ('G06F126', 'G06F1516')]
Main+other:  [('G06F116', 'G06F116'), ('G06F116', 'G06F126'), ('G06F116', 'G06F1516')]


In [45]:
### On considère une combinaison comme étant une association avec la main_IPC -- le reste ferait pas de sens?
###### POUR LA KS on a juste besoin de toutes les combinaisons
combi_direct_KS = []
combi_all_KS = []
for i in tqdm(range(len(list_applications_KS))):
    patent_path = str(list_applications_KS[i]) + '.json'
    correct_path = return_year(patent_path)
    patent_path = correct_path + patent_path
    with open(patent_path) as f:
        d = json.load(f)
        f.close()

    class_mainIPC = d['main_ipcr_label']
    non_main =  d['ipcr_labels']
    #rint(type(class_mainIPC), type(non_main))
    full_classes = [class_mainIPC] + non_main
    patent_combi = []
    list_full = []
    if len(non_main) != 0:
        for class_ipc in non_main:
            patent_combi.append((class_mainIPC, class_ipc))
        list_full = pairing(full_classes)
    combi_all_KS.extend(list_full)
    combi_direct_KS.extend(patent_combi)

100%|██████████| 507/507 [00:00<00:00, 2658.19it/s]


In [61]:
print(len(combi_all_KS), len(combi_direct_KS), combi_all_KS[0:3], combi_direct_KS[0:3])

print(combi_all_KS)
print(combi_direct_KS)

986 700 [('G06F1202', 'G06F1202'), ('G06F1516', 'G06F1516'), ('G06F116', 'G06F116')] [('G06F1202', 'G06F1202'), ('G06F1516', 'G06F1516'), ('G06F116', 'G06F116')]
[('G06F1202', 'G06F1202'), ('G06F1516', 'G06F1516'), ('G06F116', 'G06F116'), ('G06F116', 'G06F126'), ('G06F116', 'G06F1516'), ('G06F116', 'G06F126'), ('G06F116', 'G06F1516'), ('G06F126', 'G06F1516'), ('G06F3041', 'G06F3041'), ('G06F1750', 'G06F1750'), ('G06F3048', 'G06F3048'), ('G06F316', 'G06F316'), ('G06F316', 'H04R500'), ('G06F316', 'H04R500'), ('G06F3048', 'G06F3048'), ('G06F301', 'G06F301'), ('G06F301', 'G06F1730'), ('G06F301', 'G06F1730'), ('G06F1516', 'G06F1516'), ('G06F300', 'G06F300'), ('G06F1516', 'G06F1516'), ('G06F1500', 'G06F1500'), ('G06F1730', 'G06F1730'), ('G06F1730', 'G06F1730'), ('G06F3041', 'G06F3041'), ('G06F3041', 'H05K114'), ('G06F3041', 'H05K700'), ('G06F3041', 'H05K114'), ('G06F3041', 'H05K700'), ('H05K114', 'H05K700'), ('G06F2100', 'G06F2100'), ('G06F1500', 'G06F1500'), ('G06F938', 'G06F938'), ('G06F17

Il y a des pairs en double car main se retrouve dans "main_ipcr_label" et "ipcr_labels".
de plus, extend et append n'ont pas la même signification!! - en fait c'est bon, KS n'a besoin que de toutes les combis!

In [54]:
# toEval
### On considère une combinaison comme étant une association avec la main_IPC -- le reste ferait pas de sens?
combi_direct = []
combi_all = []
for i in tqdm(range(len(list_applications_toEval))):
    file = str(list_applications_toEval[i]) + '.json'
    patent_path = mypath + "/" + file
    with open(patent_path) as f:
        d = json.load(f)
        f.close()

    class_mainIPC = d['main_ipcr_label']
    non_main =  d['ipcr_labels']
    full_classes = [class_mainIPC] + non_main
    patent_combi = []
    list_full = []

    if len(full_classes) > 20:
        full_classes = full_classes[0:10]     ##### POURQUOI? on enlève les classes moins importantes?

    if len(non_main) != 0:
        for class_ipc in non_main:
            patent_combi.append((class_mainIPC, class_ipc))
        list_full = pairing(full_classes)
    combi_all.append(list_full)
    combi_direct.append(patent_combi)

100%|██████████| 19/19 [00:00<00:00, 1731.75it/s]


In [55]:
print(len(combi_all), len(combi_direct))
print(combi_all)
print(combi_direct)

19 19
[[('G06F1900', 'G06F1900'), ('G06F1900', 'G06F1718'), ('G06F1900', 'G06F1718')], [('G06F306', 'G06F306'), ('G06F306', 'G06F1342'), ('G06F306', 'G06F1208'), ('G06F306', 'G06F1328'), ('G06F306', 'G06F944'), ('G06F306', 'G06F1342'), ('G06F306', 'G06F1208'), ('G06F306', 'G06F1328'), ('G06F306', 'G06F944'), ('G06F1342', 'G06F1208'), ('G06F1342', 'G06F1328'), ('G06F1342', 'G06F944'), ('G06F1208', 'G06F1328'), ('G06F1208', 'G06F944'), ('G06F1328', 'G06F944')], [('G06F1114', 'G06F1114'), ('G06F1114', 'G06F2162'), ('G06F1114', 'G06F1730'), ('G06F1114', 'G06F2162'), ('G06F1114', 'G06F1730'), ('G06F2162', 'G06F1730')], [('G06F314', 'G06F314')], [('G06F944', 'G06F944')], [('G06F952', 'G06F952'), ('G06F952', 'G06F950'), ('G06F952', 'G06F950')], [('G06F1730', 'G06F1730')], [('G06F118', 'G06F118')], [('G06F312', 'G06F312'), ('G06F312', 'H04N132'), ('G06F312', 'H04N121'), ('G06F312', 'H04N100'), ('G06F312', 'H04N132'), ('G06F312', 'H04N121'), ('G06F312', 'H04N100'), ('H04N132', 'H04N121'), ('H04

In [89]:
#Score estimation
score_direct = []
for list_tuple in combi_direct:
    print("List: ", list_tuple)
    norm_value = len(list_tuple)
    print("Length: ", norm_value)
    score = 0
    for tuple in list_tuple:
        print("tuple: ", tuple)
        #Just so we don't count pairs of the same class
        if tuple[0] == tuple[1]:
            print('equal')
            norm_value = norm_value - 1 # update norm_value if both classes are equal
            continue
        print('not equal')
        if tuple in combi_direct_KS:        # if tuple in KS 
            print("in KS")
            continue
        else:
            score+=1   
            print("score update: ", score)                     # if tuple not in KS, novelty score = 1
    if norm_value == 0:
        score_direct.append(score)
    else:
        #Normalizing the score
        score_direct.append(score/norm_value)
    print("Norm value (end)", norm_value)
    print("Score direct", score_direct)
    print("")

List:  [('G06F1900', 'G06F1900'), ('G06F1900', 'G06F1718')]
Length:  2
tuple:  ('G06F1900', 'G06F1900')
equal
tuple:  ('G06F1900', 'G06F1718')
not equal
score update:  1
Norm value (end) 1
Score direct [1.0]

List:  [('G06F306', 'G06F306'), ('G06F306', 'G06F1342'), ('G06F306', 'G06F1208'), ('G06F306', 'G06F1328'), ('G06F306', 'G06F944')]
Length:  5
tuple:  ('G06F306', 'G06F306')
equal
tuple:  ('G06F306', 'G06F1342')
not equal
score update:  1
tuple:  ('G06F306', 'G06F1208')
not equal
score update:  2
tuple:  ('G06F306', 'G06F1328')
not equal
score update:  3
tuple:  ('G06F306', 'G06F944')
not equal
score update:  4
Norm value (end) 4
Score direct [1.0, 1.0]

List:  [('G06F1114', 'G06F1114'), ('G06F1114', 'G06F2162'), ('G06F1114', 'G06F1730')]
Length:  3
tuple:  ('G06F1114', 'G06F1114')
equal
tuple:  ('G06F1114', 'G06F2162')
not equal
in KS
tuple:  ('G06F1114', 'G06F1730')
not equal
in KS
Norm value (end) 2
Score direct [1.0, 1.0, 0.0]

List:  [('G06F314', 'G06F314')]
Length:  1
tuple: 

In [93]:
print(len(score_direct))
print(score_direct)

19
[1.0, 1.0, 0.0, 0, 0, 1.0, 0, 0, 0.6666666666666666, 1.0, 1.0, 0, 1.0, 1.0, 1.0, 1.0, 0, 1.0, 1.0]


In [90]:
#Score estimation
score_all = []
for list_tuple in combi_all:
    #print(list_tuple)
    norm_value = len(list_tuple)
    score = 0
    for tuple in list_tuple:
        #print(tuple)
        #Just so we don't count pairs of the same class
        if tuple[0] == tuple[1]:
            #print('prout')
            norm_value = norm_value - 1
            continue
        #print('prout2')
        if tuple in combi_all_KS:
            continue
        else:
            score+=1
    if norm_value == 0:
        score_all.append(score)
    else:
        #Normalizing the score
        score_all.append(score/norm_value)

In [91]:
print(len(score_all))
print(score_all)

19
[1.0, 1.0, 0.0, 0, 0, 1.0, 0, 0, 0.7777777777777778, 1.0, 1.0, 0, 1.0, 1.0, 0.8888888888888888, 1.0, 0, 1.0, 1.0]


In [67]:
df['Novel_IPC_direct'] = score_direct
df['Novel_IPC_all'] = score_all
df

,application_number,title,abstract,claims,background,summary,ipc,sec_ipc,label,Novel_IPC_direct,Novel_IPC_all
0,14994365,SYSTEMS AND METHODS FOR EMBOLISM PREDICTION US...,Systems and methods are disclosed for determin...,1. A computer-implemented method of determinin...,<SOH> BACKGROUND <EOH>Emboli are intravascular...,<SOH> SUMMARY <EOH>According to certain aspect...,G06F,"['G06F1900', 'G06F1718']",0,1.000000,1.000000
1,15028028,A new USB protocol based computer acceleration...,This study presents a new USB protocol based c...,1. The developed electronic device features a ...,<SOH> BACKGROUND OF THE INVENTION <EOH>This pr...,<SOH> SUMMARY OF THE INVENTION <EOH>The presen...,G06F,"['G06F306', 'G06F1342', 'G06F1208', 'G06F1328'...",0,1.000000,1.000000
2,15040198,Privacy Protection of Media Files For Automati...,Identifying an electronic data file for exclus...,1. A method for identifying an electronic data...,<SOH> BACKGROUND <EOH>Embodiments of the inven...,<SOH> SUMMARY <EOH>Embodiments of the inventio...,G06F,"['G06F1114', 'G06F2162', 'G06F1730']",0,0.000000,0.000000
3,15045485,METHOD AND APPARATUS FOR PROVIDING OF SCREEN M...,A method and an electronic device for providin...,1. A method for providing a screen mirroring s...,"<SOH> BACKGROUND <EOH>Digital media services, ...",<SOH> SUMMARY <EOH>To address the above-discus...,G06F,['G06F314'],1,0.000000,0.000000
4,15050101,CODE QUALITY IMPROVEMENT,Techniques enabling an end-user to specify com...,1-19. (canceled) 20. A method for improving qu...,<SOH> BACKGROUND OF THE INVENTION <EOH>Code qu...,<SOH> SUMMARY OF THE INVENTION <EOH>The instan...,G06F,['G06F944'],1,0.000000,0.000000
5,15068369,FLEXIBLE BINDING OF TASKS TO TARGET RESOURCES,A system according to one exemplary embodiment...,1. A system for managing task concurrency on c...,"<SOH> BACKGROUND <EOH>Each year, computer plat...",<SOH> BRIEF DESCRIPTION OF THE DRAWINGS <EOH>I...,G06F,"['G06F952', 'G06F950']",1,1.000000,1.000000
6,15076804,REPLICA IDENTIFICATION AND COLLISION AVOIDANCE...,A unique file-system node identification (ID) ...,1. A method for replica identification and col...,<SOH> FIELD OF THE INVENTION <EOH>The present ...,<SOH> SUMMARY OF THE INVENTION <EOH>File-syste...,G06F,['G06F1730'],1,0.000000,0.000000
7,15084569,STORAGE APPARATUS OF ELECTRONIC DEVICE,"A storage apparatus of an electronic device, t...",1. A storage apparatus of an electronic device...,<SOH> BACKGROUND <EOH>A storage apparatus moun...,<SOH> SUMMARY <EOH>According to an aspect of a...,G06F,['G06F118'],1,0.000000,0.000000
8,15093176,"INFORMATION PROCESSING APPARATUS, IMAGE FORMIN...",An information processing apparatus includes a...,1. An information processing apparatus compris...,<SOH> BACKGROUND <EOH>,<SOH> SUMMARY <EOH>According to an aspect of t...,G06F,"['G06F312', 'H04N132', 'H04N121', 'H04N100']",1,0.666667,0.777778
9,15118567,Computing Device,"A computing device includes a base member, an ...",1. A computing device comprising: a base membe...,<SOH> BACKGROUND <EOH>Computing devices such a...,<SOH> BRIEF DESCRIPTION OF THE DRAWINGS <EOH>E...,G06F,"['G06F303', 'G06F3041', 'G06F116']",0,1.000000,1.000000
